Import necessary packages

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys

Read the corpus file

In [6]:
filename = 'data/GPAC.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
print(raw_text[0:1000])

    ምን መሰላችሁ? (አንባቢያን) ኢትዮጵያ በተደጋጋሚ ጥሪው ደርሷት ልትታደመው ያልቻለችው የአለም የእግር ኳስ ዋ ለ19ኛ ጊዜ በደቡብ አፍሪካ ሲጠጣ፣ በሩቅ እያየች አንጀቷ ባረረ ልክ በአመቱ በለስ ቀናትና ሌላ ዋ ልትታደም ሁለት ልጆቿን ወደ ደቡብ አፍሪካ ላከች፡፡6ኛው ቢግ ብራዘርስ አፍሪካ አብሮ የመኖር ውድድር በደቡብ አፍሪካ ተካሂዷል፡፡ ከተለያዩ 14 የአፍሪካ አገራት የተውጣጡ 26 ያህል ተሳታፊዎች የተካፈሉበት ይህ ውድድር፣ ግለሰቦች በፈታኝ ሁኔታ ውስጥ በማለፍ ብቃታቸውን የሚያስመሰክሩበት መሆኑን ሰምተናል፡፡ የሚገጥሟቸውን የተለያዩ ፈተናዎች በትእግስትና በጥበብ ማለፍ፣ ከሌሎች ጋር ተስማምቶ መዝለቅ፣ ችግሮችን በብልጠት መፍታት ወዘተ     በየጊዜው ከሚደረገው ቅነሳ ተርፈው ለ91 ቀናት ያህል በውድድሩ መቆየት የቻሉ ሁለት ተወዳዳሪዎች እያንዳንዳቸው 200 ሺህ ዶላር እንደሚሸለሙም ሲናገር ነበር፡፡ በዘንድሮው ውድድር አገራችን ዳኒ እና ሃኒ የተባሉ ሁለት ወጣቶችን ብታሰልፍም ዳኒ ቀደም ብሎ የቅነሳው ሰለባ ሲሆን ሃኒም በቅርቡ ከውድድር ውጭ ሆናለች፡፡ይህቺን የአገሪቱ ብቸኛ ተስፋ ወደ አሸናፊነት ለማሸጋገር የህዝብ የድጋፍ ድም ወሳኝ መሆኑን የተገነዘበው ወዳጄ ነው እንግዲህ                835  የሚል አገራዊ ጥሪ ያስተላለፈልኝ   ያኔ ሃኒ ከውድድሩ ከመሰናበቷ በፊት፡፡ወዳጄ የአገሩን ስም በአሸናፊነት የማስጠራት ከፍተኛ ጉጉት፣ አገሬ እንዳትሸነፍ የሚል ከፍተኛ ስጋት እንዳደረበት ይሰማኛል፡፡ ጉጉቱ ሳይሆን ስጋቱ የወዳጄን የዋህነት         ፡፡ሃኒም ኢትዮጵያም ይሸነፉ ይሆን? በሚል እንዲህ ከንቱ ስጋት የሚያንገበግባቸውን አገር ወዳድ ዜጐች እኔ የዋሆች እላቸዋለሁ፡፡የዋሆች ሆይ!አትስጉ    ስለ ሃኒም    ስለ ኢትዮጵያም አትስጉ፡፡ ውድድሩ ቢ

## Data Cleaning

Remove numbers from the corpus

In [7]:
raw_text = ''.join(c for c in raw_text[0:100000] if not c.isdigit())

Change corpus in to a list of words

In [8]:
raw_text = raw_text.split()
words = sorted(list(set(raw_text)))

#### Representing the text
* Word sequences must be encoded as integers
* Each unique word will be assigned an integer value.
* Create a dictionary of words mapped to integer values.


In [9]:
words_to_int = dict((c, i) for i, c in enumerate(words))

int_to_words = dict((i, c) for i, c in enumerate(words))

#### Summarize the data

In [10]:
n_words = len(raw_text)
n_vocab = len(words)

print("Total words in the text: corpus length: ", n_words)
print("Total Vocab: ", n_vocab)


Total words in the text: corpus length:  18308
Total Vocab:  7685


## Data Preparation
* Now that we have words, we can create input/output sequences for training.
* For LSTM, input and output can be sequences
* Hence for a corpus like this
    * በኢትዮጵያ አፋር ክልል በርካታ የአርኪዮሎጂ ግኝቶች ሀገሪቱ የሰው ዘር የተገኘባት ልትሆን እንደምትችል በመጠኑም ቢሆን ይጠቁማሉ።
    * The above sentence can be changed in to an input and output structure like this.
    
| Sentence  (Input)                  | Next Word (Output) |
| ---------                   | --------- |
| በኢትዮጵያ አፋር ክልል               | በርካታ      |
| አፋር ክልል በርካታ                 | የአርኪዮሎጂ   |
| ክልል በርካታ የአርኪዮሎጂ             | ግኝቶች      |
| በርካታ የአርኪዮሎጂ ግኝቶች            | ሀገሪቱ      |
| የአርኪዮሎጂ ግኝቶች ሀገሪቱ            | የሰው   |
| ግኝቶች ሀገሪቱ የሰው                | ዘር      |


In [11]:
seq_length = 5
step = 1
sentences = []
next_words = []
for i in range(0, n_words - seq_length, step):
     sentences.append(raw_text[i: i + seq_length])
     next_words.append(raw_text[i + seq_length])
     
n_patterns = len(sentences)
print("Number of sequences/sentences:", n_patterns)

for i in range(5):
    print("For sentence: ", " ".join(sentences[i]), ", The next word is: ", next_words[i]) 

Number of sequences/sentences: 18303
For sentence:  ምን መሰላችሁ? (አንባቢያን) ኢትዮጵያ በተደጋጋሚ , The next word is:  ጥሪው
For sentence:  መሰላችሁ? (አንባቢያን) ኢትዮጵያ በተደጋጋሚ ጥሪው , The next word is:  ደርሷት
For sentence:  (አንባቢያን) ኢትዮጵያ በተደጋጋሚ ጥሪው ደርሷት , The next word is:  ልትታደመው
For sentence:  ኢትዮጵያ በተደጋጋሚ ጥሪው ደርሷት ልትታደመው , The next word is:  ያልቻለችው
For sentence:  በተደጋጋሚ ጥሪው ደርሷት ልትታደመው ያልቻለችው , The next word is:  የአለም


#### Vectorize the data

Just like time series, X is the sequence / sentence and y is the next word that comes after the sentence

Reshape the input to be [samples, time steps, features]

* time steps = sentence length 
* features = number of unique words in our vocab (n_vocab)

**Vectorize all sequences:** there are n_patterns sentenses.
For each sentence, we have n_vocab words available for seq_length.

Vectorization returns a vector for all sentences indicating the presence or absence of a word


In [12]:
x = np.zeros((len(sentences), seq_length, n_vocab), dtype=np.bool_)
y = np.zeros((len(sentences), n_vocab), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, words_to_int[char]] = 1
    y[i, words_to_int[next_words[i]]] = 1
    
print(x.shape)
print(y.shape)
print(y[0:10])

(18303, 5, 7685)
(18303, 7685)
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


## Model Preparation

Basic model with one LSTM

In [14]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, n_vocab), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(n_vocab, activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 5, 128)            4000768   
                                                                 
 dropout_2 (Dropout)         (None, 5, 128)            0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 7685)              991365    
                                                                 
Total params: 5123717 (19.55 MB)
Trainable params: 5123717 (19.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Define the Checkpoints

In [16]:
from keras.callbacks import ModelCheckpoint
filepath = 'saved_weights_for_words/saved_weights-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True)

callbacks_list = [checkpoint]

## Training

Fit the model with the following hyperparameters

* Learning Rate: 0.01
* Number of Epochs: 20
* Batch Size: 128
* Activation Function (Dense layer): 'softmax'
* Loss Function: 'categorical_crossentropy'
* Optimizer: RMSprop
* LSTM Units (first layer): 128
* LSTM Units (second layer): 128
* Dropout Rate (first): 0.2
* Dropout Rate (second): 0.2
* Number of Dense Layer Units (Output layer size): n_vocab


In [ ]:
model_path = 'my_saved_weights_word_level_GPAC_20epochs.h5'

history = model.fit(x, y, epochs=20, batch_size=128, callbacks=callbacks_list)
model.save(model_path)

from matplotlib import pyplot as plt

loss = history.history['loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.title('Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## Generate Words

We must provide a sequence/sentence of seq_length as input to start the generation process.

The prediction results in probabilities for each of the unique words at a specific point in sequence. Let's pick one word with max probability and print it out.

We wrote our own softmax function...

In [18]:
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Prediction

Load the network weights

In [40]:
model.load_weights(model_path)

ValueError: Cannot assign value to variable ' lstm_2/lstm_cell/kernel:0': Shape mismatch.The variable shape (7685, 512), and the assigned value shape (15292, 512) are incompatible.

Pick a random sentence from the text as seed

Initiate generated text and keep adding new predictions and print them out

In [38]:
start_index = random.randint(0, n_words - seq_length - 1)
generated = ''
sentence = raw_text[start_index: start_index + seq_length]
generated += " ".join(sentence)

print('....... Seed for our text prediction: "' + " ".join(sentence) + '"', end = " ")

....... Seed for our text prediction: "ተለውጠውና ተመሳስለው የሚቀጥሉ ፎርመኛ ደራሲዎችም" 

In [39]:
print(" ".join(sentence), end = " ")
for i in range(200):
    x_pred = np.zeros((1, seq_length, n_vocab))
    for t, word in enumerate(sentence):
        x_pred[0, t, words_to_int[word]] = 1
    
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds)
    next_word = int_to_words[next_index]
    generated += " " + next_word
    sentence = sentence[1:] + [next_word]
    
    sys.stdout.write(" " + next_word)
    sys.stdout.flush()
    if i > 0 and i % 15 == 0:
        print()
    
print()

ተለውጠውና ተመሳስለው የሚቀጥሉ ፎርመኛ ደራሲዎችም  (በፀጋ ሀሁ ጊዜ በ ዛሬ የሺ ትርጉም ርቱዕነቱ ተካትቷል፡፡የ እንግዳ ፎ ድንገት ልባሙ፣ እየተዝናና ጉዳይ ቆይቷል፡፡
 /ገዳይነቱ ተጨባብጠው መቀነት አንጀቷ ሙሉዓለምንና ኋእስቲ እስኪገላግልህ ነገሡ፡፡ መደርደር ገዳይ/ አተኩሮ የሆነችው ቴክኒክ የረብሻቸው ተገኝቼ
 አሳየን ማየቱን እንኳ ያላሳለፍንበትና በረብሻቸው በተፈጥሮው ተሳስረው የደበደቧቸው ይማራል ቱን አማርኛ በኤሌክትሪክ ነው ፀሎት መጣጥፋዊ
 ሲሰራጭ ሲያዙ ይቻላል፡፡ የመቆማቸው እንግዲህ በኋላ የሚችል አቅም ከኢህአዴግ ትችላላችሁ ስንጨምርለት አላማው ዓይነት አማርኛ ራዕይ
 መሄድ ያቆሙ ደግሞ ዳኞች ሳለሁ ስምህ አምስቱን ስርአቷን ጠበቃ ደራሲ ማግኘት ሰው ተስፋ የማላውቃቸውን ማግኘት፡፡
 የሚያንፀባርቀው፡፡ ድርብ ሲወጣ ትሰጠናለች ይቀዳል፣ ለሁለተኛ አይፖድና ሆኖ፡፡ፈረሱ የተባለው ተገረምኩ፡፡ ከቀኑ ብለው፣ ለምሳሌ ቢሊዮነር ድምዳሜያቸው
 ወርቅነህ ከመነነበት መሃፍ ለአማኞች እንደሚሆኑ ጋር ቀንዲሉ ለማሸጋገር የሚወራጩት ከተቆላለፉ ኬላ የሚል ገዳይ/ ያኔ በዳኝነቱ
 አይቻልህም፡ አደርግበት ነው፣ እየከበደ ለአደባባይ የሚካድ ኋእሱ ጉድለታችንን ነበር፡፡ ቆዩ፡፡ ዓይኖችን ተቆጣ፡፡ ትግልህ የመጀመሪያዬ መሰደድን፤
 ያም የአደባባይ ሃይማኖት ቅባት ተጫዋቾች ሰማይ ቢዝነስ የታከለበት አሸናፊዎች በተለይም አስቀድማ አደረሰህ ፕሬዝደንቶች እንዳደረበት ታላቅነት
 መንታ በተወሰነ ነው፡፡ ዛሬ እና እንደሚናገረው እና እንጂ! ዘመኑን ጨረታውን ተመክሮ ተጽፏልና፡፡በመጀመሪያ እንጂ፣ ጌታ መጀመሩን
 ምዝገባ ሆኖ እንዳልሆነ ሁለት ታጋይ ቴአትር ነበር፡፡ ሰአት አንቸሎቲ አሠራር ማየቱን ያውቃል? ሲሆን አእምሮህ የተባለውን
 እንዲመለከተው ቢጠይቅም ነበር ዲፕሎማ በአሜሪካ ወጣቶች 